<a href="https://colab.research.google.com/github/yamasaki-yu/learning/blob/master/titanic_by_optuna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://note.com/npaka/n/n88d2ccc7efcb


In [1]:
# optunaのインストール
!pip install optuna

In [2]:
# ライブラリのインポート
import optuna
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import Play, IntSlider, jslink, HBox, interactive_output

from sklearn import datasets, svm, ensemble, model_selection, metrics, preprocessing
from sklearn.model_selection import train_test_split
import sklearn.model_selection
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from statistics import mean,stdev

In [3]:
# データセットの読み込み
dataset_path = "/content/drive/My Drive/Hobby/分析関連/general_dataset/titanic/train.csv"
data = pd.read_csv(dataset_path)
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
# 欠損値のチェック
print("-- Null Check --")
for col_name in data.columns:
  n_null = data[col_name].isnull().sum()
  if n_null != 0:
    print(col_name, n_null)

# 最頻値による欠損補完
data_fillna = data.copy()
#datamart_fillna['age'].fillna("no_data", inplace=True) 
data_fillna['Age'].fillna(data["Age"].mode()[0] , inplace=True) 
data_fillna['Cabin'].fillna(data["Cabin"].mode()[0] , inplace=True) 
data_fillna['Embarked'].fillna(data["Embarked"].mode()[0] , inplace=True) 

# 欠損値の再チェック
print("-- Null Check --")
for col_name in data_fillna.columns:
  n_null = data_fillna[col_name].isnull().sum()
  if n_null != 0:
    print(col_name, n_null)


-- Null Check --
Age 177
Cabin 687
Embarked 2
-- Null Check --


In [6]:
# 不要な変数の削除
data_dropped = data_fillna.drop(["Name", "Ticket", "Cabin"], axis=1)
data_dropped.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,male,22.0,1,0,7.2500,S
1,2,1,1,female,38.0,1,0,71.2833,C
2,3,1,3,female,26.0,0,0,7.9250,S
3,4,1,1,female,35.0,1,0,53.1000,S
4,5,0,3,male,35.0,0,0,8.0500,S


In [7]:
# ダミー化
data_dummy = pd.get_dummies(data_dropped, drop_first=True)
print("dummy dataset shape : " , data_dummy.shape , "\n")

# スケーリング
mm = preprocessing.MinMaxScaler()
data_dummy = pd.DataFrame(mm.fit_transform(data_dummy),index=data_dummy.index, columns=data_dummy.columns)

data_dummy["PassengerId"] = data_dropped["PassengerId"]
data_dummy["Survived"] = data_dropped["Survived"]

data_dummy

dummy dataset shape :  (891, 10) 



,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_male,Embarked_Q,Embarked_S
0,1,0,1.0,0.271174,0.125,0.000000,0.014151,1.0,0.0,1.0
1,2,1,0.0,0.472229,0.125,0.000000,0.139136,0.0,0.0,0.0
2,3,1,1.0,0.321438,0.000,0.000000,0.015469,0.0,0.0,1.0
3,4,1,0.0,0.434531,0.125,0.000000,0.103644,0.0,0.0,1.0
4,5,0,1.0,0.434531,0.000,0.000000,0.015713,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...
886,887,0,0.5,0.334004,0.000,0.000000,0.025374,1.0,0.0,1.0
887,888,1,0.0,0.233476,0.000,0.000000,0.058556,0.0,0.0,1.0
888,889,0,1.0,0.296306,0.125,0.333333,0.045771,0.0,0.0,1.0
889,890,1,0.0,0.321438,0.000,0.000000,0.058556,1.0,0.0,0.0


In [8]:
# 検証時の条件（イテレーション100, 20回繰り返し）
n_trial=20
n_search_iter = 100

## パラメータチューニングなし

In [8]:
train_score_list = []
eval_score_list = []

# 指定した回数だけ試行を繰り返す
for i in range(0, n_trial):
  print("trial ",i)

  # データ分割
  data_train, data_eval = train_test_split(data_dummy, test_size=0.3)
  data_fit, data_valid = train_test_split(data_train, test_size=0.3)

  #モデルの構築
  model = svm.SVC()
  model.fit(data_train.drop(["PassengerId","Survived"], axis=1), data_train["Survived"])
  
  # モデル評価
  score_train = model.score(data_train.drop(["PassengerId","Survived"], axis=1), data_train["Survived"])
  score_eval = model.score(data_eval.drop(["PassengerId","Survived"], axis=1), data_eval["Survived"])
  print("score : train/eval = " , score_train, " / ", score_eval)

  # スコアの保存
  train_score_list.append(score_train)
  eval_score_list.append(score_eval)


# 繰り返しの結果を出力
print("\nstat result:")
print("  train : ave/std = " , mean(train_score_list), " / ", stdev(train_score_list) )
print("  eval  : ave/std = " , mean(eval_score_list), " / ", stdev(eval_score_list) , "\n")
print("\nlist:")
print("  train : " , train_score_list )
print("  eval  : " , eval_score_list , "\n")

trial  0
score : train/eval =  0.8073836276083467  /  0.8246268656716418
trial  1
score : train/eval =  0.812199036918138  /  0.8022388059701493
trial  2
score : train/eval =  0.8138041733547352  /  0.8097014925373134
trial  3
score : train/eval =  0.812199036918138  /  0.8134328358208955
trial  4
score : train/eval =  0.8234349919743178  /  0.7873134328358209
trial  5
score : train/eval =  0.8250401284109149  /  0.7798507462686567
trial  6
score : train/eval =  0.8057784911717496  /  0.835820895522388
trial  7
score : train/eval =  0.8234349919743178  /  0.7873134328358209
trial  8
score : train/eval =  0.8282504012841091  /  0.7723880597014925
trial  9
score : train/eval =  0.7945425361155698  /  0.8582089552238806
trial  10
score : train/eval =  0.8186195826645265  /  0.7985074626865671
trial  11
score : train/eval =  0.8057784911717496  /  0.835820895522388
trial  12
score : train/eval =  0.8202247191011236  /  0.7947761194029851
trial  13
score : train/eval =  0.8250401284109149  

## optuna

In [9]:

# 目的関数
def objective(trial):

  c = trial.suggest_loguniform("prm_C", 0.01, 100)
  kernel = trial.suggest_categorical("prm_kernel", ['linear', 'rbf', 'sigmoid'])
  gamma = trial.suggest_loguniform("prm_gamma", 0.01, 100)

  #print(c, kernel, gamma)

  # 学習
  model = svm.SVC(C=c, kernel=kernel, gamma=gamma, random_state=0)
  model.fit(data_fit.drop(["PassengerId","Survived"], axis=1), data_fit["Survived"])
   
  # モデル評価
  score_train = model.score(data_fit.drop(["PassengerId","Survived"], axis=1), data_fit["Survived"])
  score_eval = model.score(data_valid.drop(["PassengerId","Survived"], axis=1), data_valid["Survived"])

  return score_eval


train_score_list = []
eval_score_list = []
for i in range(0, n_trial):
  print("trial ",i)

  # データ分割
  data_train, data_eval = train_test_split(data_dummy, test_size=0.3)
  data_fit, data_valid = train_test_split(data_train, test_size=0.3)
  
  study = optuna.create_study(direction="maximize")
  study.optimize(objective, n_trials=n_search_iter)

  print("\nbest : params / value =  " , study.best_params, " / ", study.best_value)
  
  # 最優秀のパラメータで再学習
  prm_C = study.best_params["prm_C"]
  prm_kernel = study.best_params["prm_kernel"]
  prm_gamma = study.best_params["prm_gamma"]
  
  #モデルの構築
  model = svm.SVC(C=prm_C, kernel=prm_kernel, gamma=prm_gamma, random_state=0)
  model = svm.SVC()
  model.fit(data_train.drop(["PassengerId","Survived"], axis=1), data_train["Survived"])
  
  # モデル評価
  score_train = model.score(data_train.drop(["PassengerId","Survived"], axis=1), data_train["Survived"])
  score_eval = model.score(data_eval.drop(["PassengerId","Survived"], axis=1), data_eval["Survived"])
  print("\nscore : train/eval = " , score_train, " / ", score_eval)

  # スコアの保存
  train_score_list.append(score_train)
  eval_score_list.append(score_eval)


# 10回繰り返しの結果を出力
print("\nstat result:")
print("  train : ave/std = " , mean(train_score_list), " / ", stdev(train_score_list) )
print("  eval  : ave/std = " , mean(eval_score_list), " / ", stdev(eval_score_list) , "\n")
print("\nlist:")
print("  train : " , train_score_list )
print("  eval  : " , eval_score_list , "\n")


[I 2020-12-13 06:55:17,043] A new study created in memory with name: no-name-fc1532b5-ad27-443d-b619-f44d486317fc
[I 2020-12-13 06:55:17,072] Trial 0 finished with value: 0.5935828877005348 and parameters: {'prm_C': 0.018017289126101756, 'prm_kernel': 'rbf', 'prm_gamma': 6.535109311498345}. Best is trial 0 with value: 0.5935828877005348.
[I 2020-12-13 06:55:17,093] Trial 1 finished with value: 0.8074866310160428 and parameters: {'prm_C': 3.4351407211714906, 'prm_kernel': 'rbf', 'prm_gamma': 2.5160992931194155}. Best is trial 1 with value: 0.8074866310160428.
[I 2020-12-13 06:55:17,108] Trial 2 finished with value: 0.786096256684492 and parameters: {'prm_C': 1.6268059685202247, 'prm_kernel': 'linear', 'prm_gamma': 0.9087580348720963}. Best is trial 1 with value: 0.8074866310160428.
[I 2020-12-13 06:55:17,122] Trial 3 finished with value: 0.786096256684492 and parameters: {'prm_C': 0.0489421349657321, 'prm_kernel': 'linear', 'prm_gamma': 0.7239669793485546}. Best is trial 1 with value: 0

trial  0


[I 2020-12-13 06:55:17,254] Trial 9 finished with value: 0.8074866310160428 and parameters: {'prm_C': 0.5273833721584483, 'prm_kernel': 'rbf', 'prm_gamma': 5.062469883365112}. Best is trial 6 with value: 0.8181818181818182.
[I 2020-12-13 06:55:17,275] Trial 10 finished with value: 0.5454545454545454 and parameters: {'prm_C': 93.5060015969791, 'prm_kernel': 'sigmoid', 'prm_gamma': 50.4700740429909}. Best is trial 6 with value: 0.8181818181818182.
[I 2020-12-13 06:55:17,303] Trial 11 finished with value: 0.48128342245989303 and parameters: {'prm_C': 11.478876062646703, 'prm_kernel': 'sigmoid', 'prm_gamma': 16.28315223561893}. Best is trial 6 with value: 0.8181818181818182.
[I 2020-12-13 06:55:17,330] Trial 12 finished with value: 0.8021390374331551 and parameters: {'prm_C': 7.664610986660653, 'prm_kernel': 'rbf', 'prm_gamma': 0.07843873098204503}. Best is trial 6 with value: 0.8181818181818182.
[I 2020-12-13 06:55:17,360] Trial 13 finished with value: 0.8074866310160428 and parameters: {


best : params / value =   {'prm_C': 35.09806419993359, 'prm_kernel': 'rbf', 'prm_gamma': 4.0399222351554505}  /  0.8288770053475936

score : train/eval =  0.8073836276083467  /  0.8208955223880597
trial  1


[I 2020-12-13 06:55:20,012] Trial 7 finished with value: 0.6577540106951871 and parameters: {'prm_C': 0.02897820618920742, 'prm_kernel': 'sigmoid', 'prm_gamma': 2.6246812826259664}. Best is trial 0 with value: 0.8074866310160428.
[I 2020-12-13 06:55:20,029] Trial 8 finished with value: 0.7967914438502673 and parameters: {'prm_C': 47.374224404447325, 'prm_kernel': 'linear', 'prm_gamma': 26.80649028375469}. Best is trial 0 with value: 0.8074866310160428.
[I 2020-12-13 06:55:20,053] Trial 9 finished with value: 0.7967914438502673 and parameters: {'prm_C': 92.06794253864912, 'prm_kernel': 'sigmoid', 'prm_gamma': 0.031104103841999752}. Best is trial 0 with value: 0.8074866310160428.
[I 2020-12-13 06:55:20,079] Trial 10 finished with value: 0.7967914438502673 and parameters: {'prm_C': 0.37587924542326545, 'prm_kernel': 'rbf', 'prm_gamma': 0.2250024892502448}. Best is trial 0 with value: 0.8074866310160428.
[I 2020-12-13 06:55:20,098] Trial 11 finished with value: 0.7967914438502673 and param

...


best : params / value =   {'prm_C': 6.257905213391533, 'prm_kernel': 'rbf', 'prm_gamma': 5.662162984945764}  /  0.7807486631016043

score : train/eval =  0.8025682182985554  /  0.835820895522388
trial  18


[I 2020-12-13 06:56:04,120] Trial 8 finished with value: 0.43315508021390375 and parameters: {'prm_C': 17.86416990054161, 'prm_kernel': 'sigmoid', 'prm_gamma': 6.641125135341717}. Best is trial 4 with value: 0.7807486631016043.
[I 2020-12-13 06:56:04,139] Trial 9 finished with value: 0.5882352941176471 and parameters: {'prm_C': 0.35014153025445244, 'prm_kernel': 'sigmoid', 'prm_gamma': 94.28599296584801}. Best is trial 4 with value: 0.7807486631016043.
[I 2020-12-13 06:56:04,169] Trial 10 finished with value: 0.7807486631016043 and parameters: {'prm_C': 77.84480086120212, 'prm_kernel': 'linear', 'prm_gamma': 0.11111019510664683}. Best is trial 4 with value: 0.7807486631016043.
[I 2020-12-13 06:56:04,198] Trial 11 finished with value: 0.7807486631016043 and parameters: {'prm_C': 87.9825043059264, 'prm_kernel': 'linear', 'prm_gamma': 0.0811921220551303}. Best is trial 4 with value: 0.7807486631016043.
[I 2020-12-13 06:56:04,219] Trial 12 finished with value: 0.7807486631016043 and parame


best : params / value =   {'prm_C': 18.754277281466297, 'prm_kernel': 'rbf', 'prm_gamma': 0.05472479729131589}  /  0.7914438502673797

score : train/eval =  0.8009630818619583  /  0.8395522388059702
trial  19


[I 2020-12-13 06:56:06,622] Trial 8 finished with value: 0.786096256684492 and parameters: {'prm_C': 0.09147775284572017, 'prm_kernel': 'rbf', 'prm_gamma': 9.829942721619904}. Best is trial 2 with value: 0.8288770053475936.
[I 2020-12-13 06:56:06,644] Trial 9 finished with value: 0.8181818181818182 and parameters: {'prm_C': 5.258223392693792, 'prm_kernel': 'rbf', 'prm_gamma': 0.011325523279082409}. Best is trial 2 with value: 0.8288770053475936.
[I 2020-12-13 06:56:06,697] Trial 10 finished with value: 0.7647058823529411 and parameters: {'prm_C': 68.51673548397095, 'prm_kernel': 'rbf', 'prm_gamma': 98.44373382843763}. Best is trial 2 with value: 0.8288770053475936.
[I 2020-12-13 06:56:06,727] Trial 11 finished with value: 0.8235294117647058 and parameters: {'prm_C': 57.8249547753528, 'prm_kernel': 'linear', 'prm_gamma': 3.031712706820913}. Best is trial 2 with value: 0.8288770053475936.
[I 2020-12-13 06:56:06,747] Trial 12 finished with value: 0.8181818181818182 and parameters: {'prm_C


best : params / value =   {'prm_C': 4.059165610000006, 'prm_kernel': 'rbf', 'prm_gamma': 10.676009875405015}  /  0.8556149732620321

score : train/eval =  0.8073836276083467  /  0.8246268656716418

stat result:
  train : ave/std =  0.8112359550561797  /  0.009175104238733581
  eval  : ave/std =  0.816231343283582  /  0.020361074210994888 


list:
  train :  [0.8073836276083467, 0.812199036918138, 0.797752808988764, 0.797752808988764, 0.8105939004815409, 0.8138041733547352, 0.8170144462279294, 0.8154093097913323, 0.8186195826645265, 0.8089887640449438, 0.8314606741573034, 0.826645264847512, 0.8105939004815409, 0.8057784911717496, 0.8234349919743178, 0.8138041733547352, 0.8025682182985554, 0.8025682182985554, 0.8009630818619583, 0.8073836276083467]
  eval  :  [0.8208955223880597, 0.8134328358208955, 0.8470149253731343, 0.8470149253731343, 0.8171641791044776, 0.8059701492537313, 0.8097014925373134, 0.8059701492537313, 0.8059701492537313, 0.8208955223880597, 0.7761194029850746, 0.77985074

In [9]:
# 参考
#https://qiita.com/nanairoGlasses/items/93d764f549943d42d5e6

# サーチ用パラメータ
param_serch = {
    "C":np.arange(0.01,100, 0.01),
    "kernel":['linear', 'rbf', 'sigmoid'],
    "gamma":np.arange(0.01,100, 0.01)
    }

train_score_list = []
eval_score_list = []
for i in range(0, n_trial):
  print("trial ",i)

  # データ分割
  data_train, data_eval = train_test_split(data_dummy, test_size=0.3)
  data_fit, data_valid = train_test_split(data_train, test_size=0.3)
  
  # モデル生成
  model = RandomizedSearchCV(
      estimator=svm.SVC(), 
      n_iter=n_search_iter, 
      param_distributions=param_serch, 
      scoring="accuracy", 
      cv=2, 
      verbose=1
  )
  model.fit(data_train.drop(["PassengerId","Survived"], axis=1), data_train["Survived"])
  
  # モデル評価
  score_train = model.score(data_train.drop(["PassengerId","Survived"], axis=1), data_train["Survived"])
  score_eval = model.score(data_eval.drop(["PassengerId","Survived"], axis=1), data_eval["Survived"])
  print("score : train/eval = " , score_train, " / ", score_eval, "\n")

  # スコアの保存
  train_score_list.append(score_train)
  eval_score_list.append(score_eval)


# 10回繰り返しの結果を出力
print("\nstat result:")
print("  train : ave/std = " , mean(train_score_list), " / ", stdev(train_score_list) )
print("  eval  : ave/std = " , mean(eval_score_list), " / ", stdev(eval_score_list) , "\n")
print("\nlist:")
print("  train : " , train_score_list )
print("  eval  : " , eval_score_list , "\n")

trial  0
Fitting 2 folds for each of 100 candidates, totalling 200 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:    2.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


score : train/eval =  0.7800963081861958  /  0.8022388059701493 

trial  1
Fitting 2 folds for each of 100 candidates, totalling 200 fits


[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:    2.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


score : train/eval =  0.7913322632423756  /  0.7761194029850746 

trial  2
Fitting 2 folds for each of 100 candidates, totalling 200 fits


...

[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:    2.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


score : train/eval =  0.8747993579454254  /  0.7910447761194029 

trial  18
Fitting 2 folds for each of 100 candidates, totalling 200 fits


[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:    1.7s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


score : train/eval =  0.9149277688603531  /  0.7835820895522388 

trial  19
Fitting 2 folds for each of 100 candidates, totalling 200 fits
score : train/eval =  0.7849117174959872  /  0.7910447761194029 


stat result:
  train : ave/std =  0.8378009630818619  /  0.04120865843902416
  eval  : ave/std =  0.7953358208955223  /  0.02419020019284847 


list:
  train :  [0.7800963081861958, 0.7913322632423756, 0.9325842696629213, 0.8089887640449438, 0.797752808988764, 0.8651685393258427, 0.8475120385232745, 0.8443017656500803, 0.8619582664526485, 0.8587479935794543, 0.8507223113964687, 0.8041733547351525, 0.7961476725521669, 0.8394863563402889, 0.8218298555377207, 0.8362760834670947, 0.8443017656500803, 0.8747993579454254, 0.9149277688603531, 0.7849117174959872]
  eval  :  [0.8022388059701493, 0.7761194029850746, 0.7761194029850746, 0.7350746268656716, 0.7611940298507462, 0.8208955223880597, 0.8283582089552238, 0.7985074626865671, 0.7985074626865671, 0.8246268656716418, 0.8134328358208955, 0

[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:    1.9s finished
